<a href="https://www.kaggle.com/code/ahmetgkt/disaster-prediction-using-bert?scriptVersionId=192280306" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# build a machine learning model that predicts **which Tweets are about real disasters and which one’s aren’t.**

In [ ]:
!pip install evaluate

In [ ]:
from datasets import Dataset,load_dataset

df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df_pred = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

ds_train = Dataset.from_pandas(df_train)
ds_pred = Dataset.from_pandas(df_pred)

ds_train = ds_train.train_test_split(test_size=0.1)
ds_train["train"]

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_train = ds_train["train"].map(tokenize_function, batched=True)
tokenized_validation = ds_train["test"].map(tokenize_function, batched=True)
tokenized_test = ds_pred.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_train = tokenized_train.rename_column("target", "labels")
tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_validation = tokenized_validation.rename_column("target", "labels")
tokenized_validation.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer,TrainingArguments,AutoModelForSequenceClassification

training_args = TrainingArguments("test-trainer")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

import wandb
wandb.init(mode="disabled")
trainer.train()

In [ ]:
import evaluate

trainer.evaluate()

# Predictions


In [ ]:
predictions = trainer.predict(tokenized_test)

In [ ]:
df_pred["target"] = np.argmax(predictions.predictions, axis=-1)

df_pred = df_pred[['id', 'target']]
df_pred.to_csv("submission.csv", index=False)